In [1]:
from time import sleep

def aplicar_descuento(precio):
    sleep(2)
    precio = precio * 0.8
    return precio

def obtener_total(precio1, precio2):
    sleep(2)
    return precio1 + precio2

def obtener_precio_total(precio1, precio2):
    sleep(2)
    valor1 = aplicar_descuento(precio1)
    valor2 = aplicar_descuento(precio2)
    obtener_total(valor1, valor2)

In [3]:
import dask
from dask import delayed

In [7]:
x = delayed(aplicar_descuento(100))
y = delayed(aplicar_descuento(300))
z = delayed(obtener_precio_total)(x,y)

In [8]:
%%time
z.compute()

Wall time: 8.01 s


# Dataframes

In [10]:
import dask
import dask.dataframe as dd

In [11]:
mi_dataframe = dask.datasets.timeseries()
mi_dataframe

,id,name,x,y
npartitions=30,,,,
2000-01-01,int32,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [12]:
mi_dataframe.head()

,id,name,x,y
timestamp,,,,
2000-01-01 00:00:00,1025,Ursula,0.880671,0.788765
2000-01-01 00:00:01,1052,Bob,0.499288,0.802256
2000-01-01 00:00:02,1029,Victor,-0.725780,0.026028
2000-01-01 00:00:03,972,Ursula,0.480017,0.460402
2000-01-01 00:00:04,941,Yvonne,0.022861,-0.489006


In [13]:
df = mi_dataframe.groupby("name").y.std()
df

Dask Series Structure:
npartitions=1
    float64
        ...
Name: y, dtype: float64
Dask Name: sqrt, 67 tasks

In [19]:
df = df.persist() #si el dataframe no va a cambiar en el tiempo
resultado = df.compute()
type(resultado)

pandas.core.series.Series

In [20]:
resultado

name
Alice       0.577038
Bob         0.576332
Charlie     0.577976
Dan         0.576278
Edith       0.577237
Frank       0.576047
George      0.577348
Hannah      0.578834
Ingrid      0.576631
Jerry       0.577424
Kevin       0.576097
Laura       0.577733
Michael     0.577939
Norbert     0.578530
Oliver      0.576231
Patricia    0.577654
Quinn       0.579011
Ray         0.578718
Sarah       0.577913
Tim         0.577731
Ursula      0.576945
Victor      0.578115
Wendy       0.577745
Xavier      0.576363
Yvonne      0.578610
Zelda       0.577076
Name: y, dtype: float64

# Dask Bags

In [21]:
import dask.bag as bag

In [22]:
mi_bag = dask.bag.from_sequence(['Nestor','Nicolas','Jaime','Ignacio'])

In [23]:
mi_urls = dask.bag.from_url(['http://google.com', 'http://github.com'])

In [24]:
mi_bag

dask.bag<from_sequence, npartitions=4>

In [25]:
mi_urls

dask.bag<from_url, npartitions=2>

In [27]:
!pip install mimesis

  Created wheel for mimesis: filename=mimesis-4.1.3-py3-none-any.whl size=2849749 sha256=941ffb0d1a844299a9aa66cff5376dfa5cb9a11e71e36436b242b0c3eb7eaf60
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\0c\02\16\868723676a8615ec2e15f46b7fb51f1dd2a910afdb64a61aa6
Successfully built mimesis


In [28]:
import dask
import dask.bag
import json
import os

In [30]:
os.makedirs('directorio', exist_ok=True)

In [31]:
mi_bag = dask.datasets.make_people()

In [32]:
mi_bag

dask.bag<mimesis, npartitions=10>

In [33]:
mi_bag.map(json.dumps).to_textfiles('directorio/*.json')

['C:/Users/Administrator/Desktop/Dask/directorio/0.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/1.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/2.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/3.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/4.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/5.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/6.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/7.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/8.json',
 'C:/Users/Administrator/Desktop/Dask/directorio/9.json']

In [34]:
mi_bag.take(3)

({'age': 47,
  'name': ('Coy', 'Thompson'),
  'occupation': 'Pallet Maker',
  'telephone': '1-017-648-8032',
  'address': {'address': '1019 Saint Josephs Crescent', 'city': 'Wayne'},
  'credit-card': {'number': '4667 7154 2111 7143',
   'expiration-date': '08/23'}},
 {'age': 62,
  'name': ('Delsie', 'House'),
  'occupation': 'Shop Keeper',
  'telephone': '470-834-9260',
  'address': {'address': '643 Livingston Drung', 'city': 'Brea'},
  'credit-card': {'number': '2342 4216 7246 0061',
   'expiration-date': '08/16'}},
 {'age': 48,
  'name': ('Oretha', 'Tucker'),
  'occupation': 'Lavatory Attendant',
  'telephone': '515.886.0308',
  'address': {'address': '243 Cleveland Square', 'city': 'Socorro'},
  'credit-card': {'number': '2395 5816 3238 1297',
   'expiration-date': '08/21'}})

In [36]:
mi_bag.filter(lambda record: record['age'] > 60).take(4)

({'age': 62,
  'name': ('Delsie', 'House'),
  'occupation': 'Shop Keeper',
  'telephone': '470-834-9260',
  'address': {'address': '643 Livingston Drung', 'city': 'Brea'},
  'credit-card': {'number': '2342 4216 7246 0061',
   'expiration-date': '08/16'}},
 {'age': 66,
  'name': ('Mauro', 'Hopper'),
  'occupation': 'Circus Proprietor',
  'telephone': '1-312-655-0652',
  'address': {'address': '502 Vara Concession road', 'city': 'Christiansburg'},
  'credit-card': {'number': '2229 9555 9870 0116',
   'expiration-date': '08/19'}},
 {'age': 61,
  'name': ('Farah', 'Walls'),
  'occupation': 'Messenger',
  'telephone': '1-319-883-6209',
  'address': {'address': '100 Borica Viaduct', 'city': 'Wausau'},
  'credit-card': {'number': '5153 2587 8433 5505',
   'expiration-date': '08/20'}},
 {'age': 66,
  'name': ('Leigh', 'Flores'),
  'occupation': 'Plastics Consultant',
  'telephone': '1-657-887-5123',
  'address': {'address': '503 Tampa Green', 'city': 'Fullerton'},
  'credit-card': {'number': '

In [37]:
ocupaciones = mi_bag.map(lambda record: record['occupation'])

In [38]:
ocupaciones.take(10)

('Pallet Maker',
 'Shop Keeper',
 'Lavatory Attendant',
 'Medical Officer',
 'Airport Controller',
 'Planning Technician',
 'Special Needs',
 'Toy Trader',
 'Furniture Remover',
 'Warehouse Manager')

In [39]:
ocupaciones.count().compute()

10000

In [42]:
mis_datos = (mi_bag.filter(lambda record: record['age'] > 40).map(lambda record: record['occupation'])
             .frequencies(sort=True).topk(10,key=1))

In [43]:
mis_datos.compute()

[('Car Wash Attendant', 14),
 ('Acoustic Engineer', 12),
 ('Technical Instructor', 12),
 ('Property Manager', 12),
 ('Actress', 11),
 ('Practice Manager', 11),
 ('Archivist', 11),
 ('Employment', 11),
 ('Motor Racing', 11),
 ('Tutor', 11)]